## 라이브러리 불러오기

In [1]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
# import schedule

# XML 파싱
import xml.etree.ElementTree as ET

# 입력데이터 데이터프레임 작성
import pandas as pd
import numpy as np

# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

#EXCEL
import openpyxl

In [2]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## 기본값 설정하기

In [3]:
routeId = 204000046
year = 2020
month = 12
day = 24

## Key Unquote

In [4]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
apiKey = unquote(KEY)

## API 호출

In [5]:
# 노선정보항목조회 busrouteservice/info
# routeId의 정류장 목록을 받아옵니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
request = Request(url + queryParams)
request.get_method = lambda: 'GET'

# API를 호출하여 XML 형식으로 된 string 데이터를 변수 "oneLineXML" 에 저장합니다.
oneLineXML = urlopen(request).read().decode('utf8')
# oneLineXML

## XML 파싱

In [6]:
# xtree는 "openapi" 도메인 API 호출에서 "headerCd"에 상관없이 3개의 태그를 갖습니다.
# [comMsgHeader, msgHeader, msgBody]
xtree = ET.fromstring(oneLineXML)
# for branch in xtree:
#     print(branch)
msgBody = xtree[2]

tagList = ["stationSeq", "stationId", "stationName", "x", "y"]
stationList = []
for branch in msgBody:
    stationList.append([branch.find(tag).text for tag in tagList])

# station_df의 index가 0부터 시작하는 문제를 해결 하기 위해 rangeIndex를 지정하였습니다.
stationCount = len(stationList) # 뒤에서 다시 사용되므로 함수로 선언시 주의
rangeIndex = pd.RangeIndex(start=1, stop=stationCount+1)

station_df = pd.DataFrame(stationList, columns=tagList, index=rangeIndex)
station_df = station_df.astype({
    "stationSeq" : "int32",
    "stationId" : "int32", 
    "stationName" : "string",
    "x" : "float",
    "y" : "float"
    
})
station_df

,stationSeq,stationId,stationName,x,y
1,1,206000010,운중동먹거리촌,127.058233,37.391400
2,2,206000009,운중동푸르지오하임,127.061100,37.391933
3,3,206000658,한빛교회.월든힐스아파트,127.063967,37.392050
4,4,206000558,운중초등학교,127.066567,37.391350
5,5,206000007,운중중학교,127.073050,37.391600
6,6,206000006,운중동행정복지센터,127.077667,37.391500
7,7,206000005,뫼루니육교,127.082917,37.391067
8,8,206000004,판교원마을1.2단지.판교도서관,127.087933,37.389817
9,9,206000555,판교청소년수련관.판교종합사회복지관,127.091933,37.388983
10,10,206000003,한림아파트,127.096133,37.388217


In [7]:
'''폴더 안에 존재하는 모든(하위 폴더까지 포함) 파일의 경로를 리스트로 반환'''
def allFilePath(routeIdFolder):
    fileList = []
    for root, dirs, files in os.walk(routeIdFolder):
        for fname in files:
            full_fname = os.path.join(root, fname)
            fileList.append(full_fname)
    return fileList

In [8]:
folderPath = '/home/ubuntu/Data-Structure-Project/AWS/DATA/daily-time-table/204000046'
# allFilePath(folderPath)

## 데이터 불러오기

In [9]:
'''AWS/Data/daily-time-table폴더에 노선ID(routeId)별로 저장되어 있는 
    텍스트파일(API호출 데이터)의 경로를 존재여부에 상관없이 반환합니다.'''
def getFilePath(routeId, year, month, day):
    curRootDir = os.getcwd()

    subDir = '/AWS/DATA/daily-time-table/'

    strRouteId = str(routeId)
    routeIdFolder = curRootDir + subDir + strRouteId

    fileName = '/' + str(year) + '-' + str(month) + '-' + str(day) + '.txt'

    fileDir = routeIdFolder + fileName
    
    return fileDir

assert getFilePath(204000046, 2020, 12, 23) == '/home/ubuntu/Data-Structure-Project/AWS/DATA/daily-time-table/204000046/2020-12-23.txt'

In [10]:
'''
routeId 별 정류소 도착시간을 나타내는 2차원 배열을 생성 할 수 있습니다
마지막 정류소 번호가 필요합니다 : 경유정류소목록조회 API 서비스를 이용하여 가져옵니다.
중간 또는 처음과 끝에 발생하는 결측치를 채워야 합니다.
'''
inputPath = getFilePath(204000046, 2020, 12, 23)

itemTagList = ['endBus', 'lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
# itemTagList = ['endBus', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
itemTagList.insert(0, 'DateTime')
_df = pd.read_csv(inputPath, sep=' ', skiprows=[0], names=itemTagList)
_df = _df.drop(["Nan"], axis=1)
_df = _df.sort_values(['plateNo'])
_df = _df.dropna()
# _df = _df.drop(_df[_df["endBus"]=="API"].index)

# string을 datetime객체로 변환하기
_df['DateTime'] = pd.to_datetime(_df['DateTime'])
# dateTimeObj = datetime.datetime.strptime(_df['DateTime'], '%Y-%m-%dT%H:%M:%S.%f%z')
# _df['DateTime'] = _df.DateTime.str.split('.').str[0]
# _df['DateTime'] = _df.DateTime.str.replace('T', ' ')

_df = _df.astype({
    'endBus': 'int32', 
    'lowPlate': 'int32', 
    'plateNo': 'string', 
    'plateType': 'int32',
    'remainSeatCnt': 'int32', 
    'routeId': 'int32', 
    'stationId': 'int32', 
    'stationSeq': 'int32'
})
# _df[:10]
_df.shape

(7756, 9)

In [11]:
# stationSeq 가 뒤바뀌는 사례도 발생합니다. 
# cf) routeId = 204000046, DateTime = 20-11-17, plateNo = 경기78아1147, index = 166
# _df[_df['plateNo']=='경기78아1147'].sort_values(by='DateTime').reset_index(drop=True).iloc[160:170] 

In [12]:
# 차량 운행기록을 "plateNo"를 기준으로 구분합니다.
plateNoList = _df.groupby(['plateNo']).size().index
plateNoList

Index(['강원71바1297', '경기70아6926', '경기70아7361', '경기70아7482', '경기70아8601',
       '경기70아8629', '경기70아8789', '경기70아8880', '경기76사5089', '경기77바1097',
       '경기77바1198', '경기78아1117', '경기78아1147', '경기78아1178'],
      dtype='object', name='plateNo')

In [13]:
# 차량 번호 한개에 대해서 검사를 실행합니다.
plateNo = plateNoList[0]
temp_df =_df[_df['plateNo']==plateNo].sort_values(by='DateTime').reset_index(drop=True)
tempIndexList = []
for i in range(1, len(temp_df)):
    if(temp_df.iloc[i-1]['stationSeq'] > temp_df.iloc[i]['stationSeq']):
            tempIndexList.append(i)
            print((temp_df.iloc[i-1]['stationSeq'] , temp_df.iloc[i]['stationSeq']))
            print(i)
            
    tempIndexList.append(len(temp_df))
# display(temp_df)
# temp_df.shape

In [85]:
# 정류장의 수 만큼 행을 갖는 빈 데이터프레임을 생성합니다. 변수 rangeIndex는 [XML파싱]에서 선언되어 있습니다.
_oneDay_df = pd.DataFrame(index = rangeIndex)
# _oneDay_df

In [86]:
# 각 차량 번호를 기준으로 반복문을 수행합니다.
for plateNo in plateNoList:
    
    '''특정 차량번호와 같은(_df['plateNo'] == plateNo) 운행기록을 
       DateTime을 기준으로 오름차순으로 정렬(sort_values(['DateTime']합니다.'''
    sameBus_df = _df[_df['plateNo'] == plateNo].sort_values(['DateTime']).reset_index(drop=True)
    #display(sameBus_df)
    #print(sameBus_df.shape)
    
    '''버스는 뒤로가지 않으므로 "stationSeq"가 작아지는 인덱스 i의 리스트를 만듭니다.
       API 호출 오류로 stationSeq가 작아지는 경우가 있습니다. 이 문제를 해결하기 위해 보정계수 diff를 둡니다. 
       diff는 노선의 정류장의 총 개수(rangeIndex.stop)에 일정비율(diffRate)을 곱하여 정의합니다.'''
    indexList = []
    diffRate = 0.8
    diff = rangeIndex.stop * diffRate if ( True ) else 0
    for i in range(1,len(sameBus_df)):
        if(sameBus_df.iloc[i-1]['stationSeq'] > sameBus_df.iloc[i]['stationSeq'] + diff ):
            indexList.append(i)
            #print((sameBus_df.iloc[i-1]['stationSeq'] , sameBus_df.iloc[i]['stationSeq']))
            
    indexList.append(len(sameBus_df))
    #print(indexList)
    

    # 첫번재 인덱스, 인덱스의 리스트, 마지막 인덱스를 조합하여 호차별로 운행기록을 구분합니다.
    index = 0
    #tupleList = []
    for i in indexList:
        #tupleList.append((index, i))
        small_df = sameBus_df.iloc[index:i].groupby(by=['stationSeq']).min()
        
        # 정류소 데이터프레임에 합칩니다.
        merge_df = pd.merge(station_df, small_df, how='outer', on='stationId')
        # merge_df = merge_df.set_index(keys='stationSeq')
        
        _oneDay_df[str(plateNo)+'_'+str(index)] = merge_df['DateTime']
        index=i

# _oneDay_df

In [87]:
# _oneDay_df.isnull().sum(axis=1)

In [88]:
def get_full_data_index(_oneDay_df):
    '''
    함수 [get_full_data_index(df)]은 데이터프레임 안에서 결측치가 하나도 없는 행의 index를 반환합니다.
    시간표를 배차 시간순으로 정렬하기 위해 데이터프레임의 첫번째 행을 기준으로 열(axis=1)을 정렬합니다.
    하지만 데이터프레임의 첫번째 행을 기준으로 정렬을 하게되면 결측치로 인하여 정렬이 안되는 열이 발생합니다.
    결측치가 없는 행을 찾기 위해 제작한 함수 입니다.
    모든 행에 결측치가 존재하면 -1을 반환합니다.
    '''
    for i in _oneDay_df.index:
        nullCountSum = int(_oneDay_df[i-1:i].isnull().sum(axis=1))
        if nullCountSum == 0:
            return i
    print("모든 행에 결측치가 존재합니다.")    
    return -1

In [89]:
temp_df = _oneDay_df
lineIndex = get_full_data_index(_oneDay_df)
# ISSUE* 모든행에 결측치가 존재할때, if(lineIndex == -1), 예외처리가 필요합니다.
'''
예외 처리 방법
DatatFrame.fillna 메소드를 이용하여 결측치를 1회 채운다음
다시 함수 'get_full_data_index'를 호출한다.
단, 결측치를 처리한 데이터프레임에선 정렬 기준(columns)만 취한다.
'''
while(lineIndex < 0):
    temp_df.fillna(method='bfill', limit=1, inplace = True)
    lineIndex = get_full_data_index(_oneDay_df)

In [90]:
'''
결측치가 하나도 없는 행(lineIndex)를 기준으로 데이터프레임의 열(columns)를 정렬한다.
정렬 후 데이터 프레임의 열의 순서를 변수 'sortedDfColumns' 리스트에 저장한다.
정렬된 컬럼 리스트로 _oneDay_df를 정렬하고 이를 oneDay_df에 저장한다.
'''
sortedDfColumns = temp_df.sort_values(by=lineIndex, axis=1).columns
oneDay_df = _oneDay_df[sortedDfColumns]
# oneDay_df

## 앞차와 뒷차의 운행기록을 기준으로 결측치를 채운다.

In [163]:
'''
01. 데이터프레임의 행 하나(시리즈)를 가져온다.
02. 시리즈의 각 데이터에서 기준시간을 빼서 timedelta 타입으로 변환한다.
    메소드 'interpolate'를 호출하기 위해선 데이터 타입이 'timedelta'여야 한다. 
03. Series.interpolate()를 호출하여 결측치를 채운다.
04. timedelta 타입 데이터를 datetime 타입으로 형 변환한다.
05. 시리즈를 반환한다. 함수 종료.
'''
isShowInfo = True

# 1.
datetimeArr = oneDay_df.iloc[0]

# 2. 시리즈(np.array)에서 기준시간을 뺀다. 데이터 타입이 timedelta로 바뀐다.
sampleTime = np.datetime64('1970-01-01')
timedeltaArr = datetimeArr - sampleTime
timedeltaSeries = pd.Series(timedeltaArr)
totalSecondsSeries = timedeltaSeries.dt.total_seconds()

# *데이터 확인
if (isShowInfo):
    # Count NaT
    beforeNullCnt = timedeltaSeries.isnull().sum()
    print(f"결측치 처리 이전 NaT Cnt:{beforeNullCnt:3d}개")
    
    # Display numpy array dataframe style
    tempDfTDS = pd.DataFrame(datetimeArr).transpose()
    display(tempDfTDS)

# 3. Series.interpolate() 메소드를 호출하여 결측치를 채운다
newTimedeltaSeries = totalSecondsSeries.interpolate(limit=1)

# 4. 
newDatetimeSeries = pd.to_datetime(newTimedeltaSeries, unit='s')
newDatetimeSeries = newDatetimeSeries.dt.strftime('%Y-%m-%d %H:%M:%S')

# *데이터 확인
if (isShowInfo):
    # Count NaT
    beforeNullCnt = newTimedeltaSeries.isnull().sum()
    print(f"결측치 처리 이전 NaT Cnt:{beforeNullCnt:3d}개")
    
    # Display numpy array dataframe style
    tempDfTDS = pd.DataFrame(newDatetimeSeries).transpose()
    display(tempDfTDS)

결측치 처리 이전 NaT Cnt: 20개


,경기70아8880_0,경기70아8601_0,경기78아1178_0,경기70아8629_0,경기77바1198_0,경기70아8789_0,경기70아7482_0,경기78아1147_0,경기78아1117_0,경기70아7361_0,경기76사5089_0,경기77바1097_0,경기70아8880_81,경기70아6926_0,강원71바1297_0,경기70아8601_81,경기78아1178_94,경기70아8629_94,경기77바1198_103,경기70아8789_100,경기70아7482_100,경기78아1147_104,경기78아1117_93,경기70아7361_103,경기70아6926_106,경기77바1097_111,경기70아8880_195,경기70아8601_183,경기78아1178_194,경기70아8629_192,경기77바1198_206,경기70아8789_197,경기70아6926_202,경기70아7482_196,경기78아1147_197,경기78아1117_188,경기70아7361_201,경기77바1097_215,경기70아8880_293,경기70아8601_281,경기78아1178_291,경기70아8629_288,경기77바1198_301,경기70아8789_290,경기70아6926_300,경기70아7482_302,경기78아1147_298,경기78아1117_290,경기70아7361_301,경기77바1097_318,경기70아8880_396,경기70아8601_401,경기78아1178_394,경기70아8629_409,경기77바1198_413,경기70아8789_400,경기70아6926_417,경기70아7482_424,경기78아1147_413,경기78아1117_414,경기70아7361_424,경기77바1097_444,경기70아8880_525,경기70아8601_544,경기78아1178_524,경기70아8629_540,경기77바1198_544,경기70아8789_533,경기70아6926_542,경기70아7482_549,경기78아1147_526,경기78아1117_523,경기70아7361_562,경기77바1097_554
1,2020-12-23 05:01:13,NaT,2020-12-23 05:40:56,2020-12-23 05:57:48,2020-12-23 06:10:42,2020-12-23 06:21:38,NaT,2020-12-23 06:42:29,NaT,2020-12-23 07:01:23,NaT,2020-12-23 07:16:15,NaT,NaT,NaT,NaT,NaT,2020-12-23 08:15:47,2020-12-23 08:25:43,NaT,2020-12-23 08:48:30,2020-12-23 09:01:22,2020-12-23 09:16:13,2020-12-23 09:31:05,2020-12-23 09:45:57,2020-12-23 10:00:50,2020-12-23 10:16:41,NaT,2020-12-23 10:46:26,2020-12-23 11:01:21,2020-12-23 11:16:12,2020-12-23 11:31:06,NaT,2020-12-23 12:00:54,2020-12-23 12:20:45,NaT,2020-12-23 13:01:27,2020-12-23 13:21:20,2020-12-23 13:41:13,2020-12-23 14:02:05,2020-12-23 14:20:56,2020-12-23 12:37:38,2020-12-23 15:01:41,2020-12-23 15:16:36,2020-12-23 15:31:29,2020-12-23 15:46:25,2020-12-23 16:01:20,NaT,2020-12-23 16:31:09,2020-12-23 16:46:02,2020-12-23 17:01:55,2020-12-23 17:11:52,2020-12-23 17:21:49,2020-12-23 17:30:46,2020-12-23 17:41:42,NaT,2020-12-23 18:04:35,2020-12-23 18:16:30,2020-12-23 18:31:24,NaT,2020-12-23 19:01:15,2020-12-23 19:16:10,2020-12-23 19:31:06,2020-12-23 19:47:01,2020-12-23 20:00:54,2020-12-23 20:20:47,2020-12-23 20:41:42,NaT,NaT,2020-12-23 21:41:18,NaT,NaT,2020-12-23 22:50:55,2020-12-23 23:12:48


결측치 처리 이전 NaT Cnt:  6개


,경기70아8880_0,경기70아8601_0,경기78아1178_0,경기70아8629_0,경기77바1198_0,경기70아8789_0,경기70아7482_0,경기78아1147_0,경기78아1117_0,경기70아7361_0,경기76사5089_0,경기77바1097_0,경기70아8880_81,경기70아6926_0,강원71바1297_0,경기70아8601_81,경기78아1178_94,경기70아8629_94,경기77바1198_103,경기70아8789_100,경기70아7482_100,경기78아1147_104,경기78아1117_93,경기70아7361_103,경기70아6926_106,경기77바1097_111,경기70아8880_195,경기70아8601_183,경기78아1178_194,경기70아8629_192,경기77바1198_206,경기70아8789_197,경기70아6926_202,경기70아7482_196,경기78아1147_197,경기78아1117_188,경기70아7361_201,경기77바1097_215,경기70아8880_293,경기70아8601_281,경기78아1178_291,경기70아8629_288,경기77바1198_301,경기70아8789_290,경기70아6926_300,경기70아7482_302,경기78아1147_298,경기78아1117_290,경기70아7361_301,경기77바1097_318,경기70아8880_396,경기70아8601_401,경기78아1178_394,경기70아8629_409,경기77바1198_413,경기70아8789_400,경기70아6926_417,경기70아7482_424,경기78아1147_413,경기78아1117_414,경기70아7361_424,경기77바1097_444,경기70아8880_525,경기70아8601_544,경기78아1178_524,경기70아8629_540,경기77바1198_544,경기70아8789_533,경기70아6926_542,경기70아7482_549,경기78아1147_526,경기78아1117_523,경기70아7361_562,경기77바1097_554
1,2020-12-23 05:01:13,2020-12-23 05:21:04,2020-12-23 05:40:56,2020-12-23 05:57:48,2020-12-23 06:10:42,2020-12-23 06:21:38,2020-12-23 06:32:03,2020-12-23 06:42:29,2020-12-23 06:51:56,2020-12-23 07:01:23,2020-12-23 07:08:49,2020-12-23 07:16:15,2020-12-23 07:26:10,NaN,NaN,NaN,NaN,2020-12-23 08:15:47,2020-12-23 08:25:43,2020-12-23 08:37:06,2020-12-23 08:48:30,2020-12-23 09:01:22,2020-12-23 09:16:13,2020-12-23 09:31:05,2020-12-23 09:45:57,2020-12-23 10:00:50,2020-12-23 10:16:41,2020-12-23 10:31:33,2020-12-23 10:46:26,2020-12-23 11:01:21,2020-12-23 11:16:12,2020-12-23 11:31:06,2020-12-23 11:46:00,2020-12-23 12:00:54,2020-12-23 12:20:45,2020-12-23 12:41:06,2020-12-23 13:01:27,2020-12-23 13:21:20,2020-12-23 13:41:13,2020-12-23 14:02:05,2020-12-23 14:20:56,2020-12-23 12:37:38,2020-12-23 15:01:41,2020-12-23 15:16:36,2020-12-23 15:31:29,2020-12-23 15:46:25,2020-12-23 16:01:20,2020-12-23 16:16:14,2020-12-23 16:31:09,2020-12-23 16:46:02,2020-12-23 17:01:55,2020-12-23 17:11:52,2020-12-23 17:21:49,2020-12-23 17:30:46,2020-12-23 17:41:42,2020-12-23 17:53:08,2020-12-23 18:04:35,2020-12-23 18:16:30,2020-12-23 18:31:24,2020-12-23 18:46:19,2020-12-23 19:01:15,2020-12-23 19:16:10,2020-12-23 19:31:06,2020-12-23 19:47:01,2020-12-23 20:00:54,2020-12-23 20:20:47,2020-12-23 20:41:42,2020-12-23 21:01:34,NaN,2020-12-23 21:41:18,2020-12-23 22:04:30,NaN,2020-12-23 22:50:55,2020-12-23 23:12:48


## 배차간격을 확인하고 오류데이터를 NaN으로 변환한다.

In [176]:
newTimedeltaSeries[42]

1608735701.0

In [195]:
'''
앞차와 뒷차의 배차간격을 저장하는 리스트를 반환한다. 
01. 첫번째 데이터를 예외처리한다. 
    배차간격의 수는 배차수 - 1 이므로 첫번째 데이터를 0으로 하거나 첫차시간을 기록한다.
02. 앞차(prevData) 또는 뒷차(nextData)가 NaT일때 예외처리 한다.
    next
03. 오류데이터(운행 기록 시간이 작아짐)를 예외처리 한다.
'''
prevData = 0
intervalList = []
i = 0

for nextData in newTimedeltaSeries:
    i += 1
    interval = -9999
    if(prevData):
        interval = nextData - prevData
        if(interval<0):
            '''
            오류데이터를 시리즈 데이터에서 NaT로 초기화하는 과정이 필요하다.
            함수로 사용한다면 시리즈 데이터의 값이 바뀔수 있도록 접근해야 한다.
            '''
            newTimedeltaSeries.iloc[i] = float('NaN') # 바뀌지 않는것 같다...
            interval = float('NaN')
            intervalList.append(interval)
            prevData = 0
            continue
        interval /= 60
        intervalList.append(interval)
    else:
        interval = float('NaN')
        intervalList.append(interval)
#     print(f"{i:3d}\tPREV: {prevData:10.0f}\tNEXT: {nextData:10.0f}\tINTERVAL: {interval:7.1f}")
    
    prevData = nextData

intervalList = np.round(intervalList, 0)
print(f"INT LIST의 데이터 수:{len(intervalList):3d}\n", intervalList)

INT LIST의 데이터 수: 74
 [nan 20. 20. 17. 13. 11. 10. 10.  9.  9.  7.  7. 10. nan nan nan nan nan
 10. 11. 11. 13. 15. 15. 15. 15. 16. 15. 15. 15. 15. 15. 15. 15. 20. 20.
 20. 20. 20. 21. 19. nan nan nan 15. 15. 15. 15. 15. 15. 16. 10. 10.  9.
 11. 11. 11. 12. 15. 15. 15. 15. 15. 16. 14. 20. 21. 20. nan nan 23. nan
 nan 22.]


In [92]:
# datetime 객체를 string 객체로 변환합니다. 시간을 가지고 계산해야 하는 일은 이전에 실행하세요.
oneDayString_df = oneDay_df.astype('string')
for column in oneDayString_df.columns:
    oneDayString_df[column] = oneDayString_df[column].str.slice(start=10, stop=16)
# print(oneDayString_df.shape)
    
# [column명 : 차량번호] station_df 와 oneDay_df를 합칩니다. datetime이 아닌 다른 타입의 오브젝트가 행에 붙게 되므로 지금부터는 편집이 어렵습니다.
_finalOneDay_df = pd.concat([station_df,oneDayString_df], axis=1)
_finalOneDay_df.set_index('stationSeq', drop=True, inplace=True)
_finalOneDay_df.shape

(55, 78)

In [93]:
# 정렬된 column에 출차 순서를 배정합니다. (1호차, 2호차, ...)
oneDayString_df.columns = oneDayString_df.columns

In [94]:
# [column명 : n호차] station_df 와 oneDay_df를 합칩니다. datetime이 아닌 다른 타입의 오브젝트가 행에 붙게 되므로 지금부터는 편집이 어렵습니다.
finalOneDay_df = pd.concat([station_df,oneDayString_df], axis=1)
df = finalOneDay_df.set_index('stationSeq', drop=True)

In [ ]:
html = df.to_html()
# writedata.py
f = open("test.html", 'w')

f.write(html)
f.close()

In [ ]:
print(f"\n{{location}}\n\n")

In [ ]:
for i in range(len(station_df)):
    location = (station_df.iloc[i]['y'], station_df.iloc[i]['x'])
    stationName = station_df.iloc[i]['stationName']
#     print("{content: '<div>", stationName, "</div>',latlng: new kakao.maps.LatLng", location, "},", sep="")

## 일일 시간표 데이터 csv로 내보내기

In [ ]:
rootPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/DailyTimeTable/' + str(routeId)
createFolder(rootPath)
outputPath = rootPath + '/' + str(routeId) + '_' + str(year)[-2:] + '-' + str(month) + '-' + str(day) + '.csv'
df.to_csv(outputPath)

In [ ]:
print(df.to_markdown())

# CSV 파일을 읽어서 EXCEL파일로 내보낸다 : 블로그 게시 목적

## 일일 시간표 데이터 csv 읽어오기

In [ ]:
df = pd.read_csv(outputPath)

## 노선번호 받아오기

In [ ]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
apiKey = unquote(KEY)

# 노선정보항목조회 busrouteservice/info
# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
# print(url+queryParams)

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
oneLineXML = urlopen(request).read().decode('utf8')
# print(oneLineXML)

xtree = ET.fromstring(oneLineXML)
resultCode = int(xtree[1].find("resultCode").text)
msgBody = xtree[2]

busRouteInfoItem = msgBody[0]
i=0
for info in busRouteInfoItem:
    print(f"{i} {info.tag} : {info.text}")
    i+=1

In [ ]:
routeName = busRouteInfoItem.find("routeName").text

## Excel로 내보내기

In [ ]:
rootPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/ExcelExport/'
excelFileName = str(routeName) + '.xlsx'
filePath = rootPath + excelFileName

createFolder(rootPath)
print(filePath)

### 배포용 DataFrame 수정

In [ ]:
export_df = df.set_index('stationName').drop('stationId', axis=1)

### 내보내기

In [ ]:
#-- write an object to an Excel sheet using pd.DataFrame.to_excel()
export_df.to_excel(filePath, # directory and file name to write
            sheet_name = '평일', 
            na_rep = 'NaN', 
            float_format = "%.2f", 
            header = True, 
            #columns = ["group", "value_1", "value_2"], # if header is False
            index = True, 
            index_label = "정류장", 
            startrow = 0, 
            startcol = 0, 
            engine = 'openpyxl', 
            freeze_panes = (1, 1)
            )

## Excel 접근하여 수정하기
https://naakjii.tistory.com/56

In [ ]:
wb = openpyxl.load_workbook(filePath)
sheet = wb['평일']

In [ ]:
sheet.max_column, sheet.max_row

In [ ]:
sheet.column_dimensions['A'].width = 14

In [ ]:
for cell in sheet['A']:
    cell.alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center')
#     cell.fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=openpyxl.styles.colors.Color(rgb='00FF00'))

In [ ]:
wb.save(filePath)

In [ ]:
wb.close()